**The plan**
1. get data of stocks and news
2. Save dataset for training
3. Build 2 models
    * selector_model
    * trader_model
* selector_model - it is seperately trained to select best performing companies
* trader_model - includes both LSTM and PPO which are individually trained (with one stock)
4. After training, evaluate model with Sharpe ratio and Sorinto ratio

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

def compute_technical_indicators(df):
    """
    Given a DataFrame with OHLCV data for a single ticker,
    compute the following technical indicators:
      - EMA_12, EMA_26, MACD, Signal
      - RSI (14-day)
      - CCI (14-day)
      - ADX (14-day)
    """
    df = df.copy()

    # Remove columns if they exist
    for col in ['Dividends', 'Stock Splits']:
        if col in df.columns:
            df = df.drop(col, axis=1)

    # Calculate Typical Price (TP) for CCI
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3

    # EMAs for MACD
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # RSI calculation (14-day)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # CCI calculation (14-day)
    n = 14
    df['SMA_TP'] = df['TP'].rolling(n).mean()
    df['MAD_TP'] = df['TP'].rolling(n).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
    df['CCI'] = (df['TP'] - df['SMA_TP']) / (0.015 * df['MAD_TP'])

    # ADX calculation (14-day)
    df['+DM'] = df['High'].diff()
    df['-DM'] = df['Low'].diff()
    df['+DM'] = np.where((df['+DM'] > df['-DM']) & (df['+DM'] > 0), df['+DM'], 0)
    df['-DM'] = np.where((df['-DM'] > df['+DM']) & (df['-DM'] > 0), df['-DM'], 0)
    df['TR'] = np.maximum(df['High'] - df['Low'],
                          np.maximum(abs(df['High'] - df['Close'].shift(1)),
                                     abs(df['Low'] - df['Close'].shift(1))))
    df['ATR'] = df['TR'].rolling(n).mean()
    df['+DI'] = (df['+DM'].rolling(n).mean() / df['ATR']) * 100
    df['-DI'] = (df['-DM'].rolling(n).mean() / df['ATR']) * 100
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].rolling(n).mean()

    # Select required columns and backfill to handle NaN values
    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume',
                        'EMA_12', 'EMA_26', 'MACD', 'Signal', 'RSI', 'CCI', 'ADX']
    df_final = df[required_columns].bfill()

    return df_final

def get_individual_ticker_metrics(period="1mo"):
    # List of Dow Jones tickers
    dow_jones_tickers = [
        "MMM", "AXP", "AMGN", "AAPL", "BA", "CAT", "CVX", "CSCO", "KO",
        "DOW", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "MCD",
        "MRK", "MSFT", "NKE", "PG", "CRM", "TRV", "UNH", "V", "VZ",
        "WBA", "WMT", "DIS"
    ]

    # Download data for all tickers (results in a DataFrame with MultiIndex columns)
    data = yf.download(tickers=dow_jones_tickers, period=period)

    # Dictionary to store each company's metrics data
    ticker_dict = {}

    for ticker in dow_jones_tickers:
        try:
            # Extract data for the ticker using xs (cross-section) on the ticker level
            df_ticker = data.xs(ticker, level=1, axis=1)
            df_metrics = compute_technical_indicators(df_ticker)
            ticker_dict[ticker] = df_metrics
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return ticker_dict

# Retrieve individual metrics data for each company
individual_data = get_individual_ticker_metrics(period="10y")

df_all = pd.concat(individual_data, axis=1)

# Step 2: Name the levels of the MultiIndex columns.
df_all.columns.names = ['Ticker', 'Metric']

# Step 3: Swap the levels so that the top level becomes the metric and the sub-level becomes the ticker.
df_all = df_all.swaplevel(axis=1)

# (Optional) Sort the columns so that each metric is grouped together.
df_all = df_all.sort_index(axis=1, level=0)

# Now, df_all's columns are in the format:
#   Open:   AAPL, MSFT, ...
#   Close:  AAPL, MSFT, ...
#   etc.
print(df_all.head())
print(df_all.columns)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  30 of 30 completed


Metric            ADX                                                         \
Ticker           AAPL       AMGN        AXP         BA        CAT        CRM   
Date                                                                           
2015-02-25  36.528085  11.538526  25.726826  18.468305  19.214185  52.699942   
2015-02-26  36.528085  11.538526  25.726826  18.468305  19.214185  52.699942   
2015-02-27  36.528085  11.538526  25.726826  18.468305  19.214185  52.699942   
2015-03-02  36.528085  11.538526  25.726826  18.468305  19.214185  52.699942   
2015-03-03  36.528085  11.538526  25.726826  18.468305  19.214185  52.699942   

Metric                                                 ...    Volume  \
Ticker           CSCO        CVX       DIS        DOW  ...       MRK   
Date                                                   ...             
2015-02-25  43.388103  37.313255  19.79816  11.768034  ...   8861050   
2015-02-26  43.388103  37.313255  19.79816  11.768034  ...  12040577   

In [ ]:
# df_all# Save the DataFrame to a CSV file in your current working directory
# df_all.to_csv("combined_ticker_data.csv", index=True)

In [16]:
import requests
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import time
import sys  # Import sys to allow exiting the process

# Replace with your Guardian API key
api_key = "YOUR_API_KEY"

# Mapping of Dow Jones tickers to full company names
company_mapping = {
    "MMM": "3M",
    "AXP": "American Express",
    "AMGN": "Amgen",
    "AAPL": "Apple",
    "BA": "Boeing",
    "CAT": "Caterpillar",
    "CVX": "Chevron",
    "CSCO": "Cisco Systems",
    "KO": "Coca-Cola",
    "DOW": "Dow",
    "GS": "Goldman Sachs",
    "HD": "Home Depot",
    "HON": "Honeywell",
    "IBM": "IBM",
    "INTC": "Intel",
    "JNJ": "Johnson & Johnson",
    "JPM": "JPMorgan Chase",
    "MCD": "McDonald's",
    "MRK": "Merck",
    "MSFT": "Microsoft",
    "NKE": "Nike",
    "PG": "Procter & Gamble",
    "CRM": "Salesforce",
    "TRV": "Travelers",
    "UNH": "UnitedHealth Group",
    "V": "Visa",
    "VZ": "Verizon",
    "WBA": "Walgreens Boots Alliance",
    "WMT": "Walmart",
    "DIS": "Disney"
}

# Define the 10-year date range
end_date = datetime.datetime.today()
start_date = end_date - relativedelta(years=10)
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Dictionary to hold DataFrames of articles for each company
company_articles = {}

for ticker, company in company_mapping.items():
    print(f"Fetching articles for {company} ({ticker}) ...")
    page = 1
    articles = []
    
    while True:
        url = "https://content.guardianapis.com/search"
        params = {
            "api-key": "1e78027b-d07c-4e35-9a0a-8f1d2b4e5549", #"75e3c8c0-28e6-4166-961c-a72883c8ea3a",
            "from-date": start_date_str,
            "to-date": end_date_str,
            "q": company,         # Use the full company name as query
            "page": page,
            "page-size": 50       # Adjust page-size as needed (subject to API limits)
        }
        response = requests.get(url, params=params)
        
        # Check for API rate limit exceeded
        if response.status_code == 429:
            print("API rate limit exceeded. Stopping process.")
            sys.exit()
        elif response.status_code != 200:
            print(f"Error fetching data for {company}: HTTP {response.status_code}")
            break
        
        data = response.json()
        resp = data.get("response", {})
        results = resp.get("results", [])
        
        if not results:
            break
        
        # Process each article (each remains as a separate row)
        for item in results:
            pub_date = item.get("webPublicationDate", None)
            if pub_date:
                pub_date = pub_date[:10]  # Extract YYYY-MM-DD
            articles.append({
                "pub_date": pub_date,
                "webTitle": item.get("webTitle", ""),
                "webUrl": item.get("webUrl", "")
            })
        
        current_page = resp.get("currentPage", 1)
        total_pages = resp.get("pages", 1)
        if current_page >= total_pages:
            break
        else:
            page += 1
            time.sleep(0.2)  # Pause to avoid rate limits
    
    # Save the articles as a DataFrame for this company
    company_articles[company] = pd.DataFrame(articles)

# --------------------------
# Option: Long Format DataFrame
# Each row is one article and includes a 'company' column.
long_df = pd.concat([
    df.assign(company=company) for company, df in company_articles.items()
], ignore_index=True)

print("Long Format DataFrame (one row per article):")
print(long_df.head(10))


Fetching articles for 3M (MMM) ...
Fetching articles for American Express (AXP) ...
Fetching articles for Amgen (AMGN) ...
Fetching articles for Apple (AAPL) ...
Fetching articles for Boeing (BA) ...
Fetching articles for Caterpillar (CAT) ...
Fetching articles for Chevron (CVX) ...
Fetching articles for Cisco Systems (CSCO) ...
Fetching articles for Coca-Cola (KO) ...
Fetching articles for Dow (DOW) ...
Fetching articles for Goldman Sachs (GS) ...
Fetching articles for Home Depot (HD) ...
API rate limit exceeded. Stopping process.


SystemExit: 

c:\ProgramData\anaconda3\envs\RL_project\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
